# **Install requrements**

In [ ]:
!pip install  transformers datasets

# **import libraries**

In [30]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np
import torch
import math

# **Load and split the dataset**

In [31]:
# Load and split the dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_test_split = dataset["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]


# **Initialize tokenizer**

In [34]:
# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Add padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [35]:
# Define a function to tokenize the text
def tokenize_function(examples):
    # Tokenize the text and return input_ids, attention_mask, and labels
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [36]:
# Apply the tokenizer to the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# **Load model**

In [ ]:
# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# **training arguments**

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch] -U

In [ ]:
!pip show accelerate

In [38]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",           # Directory to save the model
    eval_strategy="steps",            # Evaluate the model every 'eval_steps'
    save_strategy="steps",            # Save the model every 'save_steps'
    learning_rate=5e-5,               # Learning rate for training
    per_device_train_batch_size=4,    # Batch size for training
    per_device_eval_batch_size=4,     # Batch size for evaluation
    num_train_epochs=3,               # Number of training epochs
    weight_decay=0.01,                # Weight decay for regularization
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,                 # Log every 'logging_steps'
    eval_steps=500,                   # Evaluate every 'eval_steps'
    save_steps=500,                   # Save every 'save_steps'
    save_total_limit=2,               # Limit the total amount of checkpoints
    load_best_model_at_end=True,      # Load the best model at the end of training
    metric_for_best_model="loss",     # The metric to use to compare models
    greater_is_better=False           # Whether a greater metric is better
)


# **Metrics**

In [39]:
# Define custom metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = (predictions == labels).mean()

    # Calculate perplexity
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    perplexity = math.exp(loss)

    return {
        "accuracy": accuracy,
        "perplexity": perplexity,
        "loss": loss.item()
    }

# **Trainer**

In [ ]:
# Use DataCollatorWithPadding to handle dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,                          # The model to train
    args=training_args,                   # Training arguments
    train_dataset=tokenized_train_dataset,  # Training dataset
    eval_dataset=tokenized_test_dataset,  # Evaluation dataset
    compute_metrics=compute_metrics,      # Custom metrics
    data_collator=data_collator           # Dynamic padding
)

# Train the model
trainer.train()


# **Evaluation**

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation results:", results)
